# Problem Formation

Given a Pattern String as an input, we want to know if it contains dark pattern in it. We use a balanced dataset cotaining all the instances in the Princeton dataset which are all dark patterns, and the instances in the 'normie.csv' file which are labeled as NOT dark patterns. Hence we have a balanced dataset consisting of pattern strings with dark pattern and without park patterns.

Then we use this labeled dataset to build and train supervised machine learning models, and select most suitable ones for our project.

----


In [1]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer

# Bernoulli Naive Bayes (Similar as  MultinomialNB), this classifier is suitable for discrete data. The difference between MultinomialNB and BernoulliNB is that while  MultinomialNB works with occurrence counts, BernoulliNB is designed for binary/boolen features, which means in the case of text classification, word occurrence vectores (rather than word count vectors) may be more suitable to be used to train and use this classifier.
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

# Evaluation metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score

# joblib is a set of tools to provide lightweight pipelining in Python. It provides utilities for saving and loading Python objects that make use of NumPy data structures, efficiently.
import joblib

import matplotlib.pyplot as plt
# import seaborn as sns

## Data Exploration

---
Import the merged dataset, and explore the dataset.

In [2]:
data = pd.read_csv('enriched_data.csv')

In [3]:
data.head(5)

,Pattern String,classification
0,Price from low to high,1
1,Price from high to low,1
2,Price High - Low,1
3,Price Low - High,1
4,Cart is currently empty,1


---
`check the dataset information`

There are 3706 NOT NULL instances of pattern strings in the dataset.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3706 entries, 0 to 3705
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Pattern String  3706 non-null   object
 1   classification  3706 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 58.0+ KB


In [5]:
# check the distribution of the target value --- classification.

print('Distribution of the tags:\n{}'.format(data['classification'].value_counts()))

Distribution of the tags:
1    2793
0     913
Name: classification, dtype: int64


In [6]:
# Change the label into strings

data['classification'].replace({0:'Dark',1:'Not_Dark'}, inplace = True)

print(data.head(5))

print('\nDistribution of the tags:\n{}'.format(data['classification'].value_counts()))

            Pattern String classification
0   Price from low to high       Not_Dark
1   Price from high to low       Not_Dark
2         Price High - Low       Not_Dark
3         Price Low - High       Not_Dark
4  Cart is currently empty       Not_Dark

Distribution of the tags:
Not_Dark    2793
Dark         913
Name: classification, dtype: int64


In [7]:
# For later training the model, we should remove the duplicate input to reduce overfitting.

data = data.drop_duplicates(subset="Pattern String")

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3706 entries, 0 to 3705
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Pattern String  3706 non-null   object
 1   classification  3706 non-null   object
dtypes: object(2)
memory usage: 86.9+ KB


In [8]:
print(data.head(5))

print('\nDistribution of the tags:\n{}'.format(data['classification'].value_counts()))

            Pattern String classification
0   Price from low to high       Not_Dark
1   Price from high to low       Not_Dark
2         Price High - Low       Not_Dark
3         Price Low - High       Not_Dark
4  Cart is currently empty       Not_Dark

Distribution of the tags:
Not_Dark    2793
Dark         913
Name: classification, dtype: int64


---
## Data Preparation

In [9]:
Y = data['classification']
X = data['Pattern String']

---
`Encode the target vales into integers` --- 'classification'

In [10]:
encoder = LabelEncoder()
encoder.fit(Y)
y = encoder.transform(Y)
y.shape

(3706,)

In [11]:
# check the mapping of encoding results (from 0 to 1 representing 'Dark', 'Not Dark')

list(encoder.classes_)

['Dark', 'Not_Dark']

In [12]:
# Check the frequency distribution of the training pattern classification with pattern classification names.

(unique, counts) = np.unique(Y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[['Dark' 913]
 ['Not_Dark' 2793]]


In [13]:
# Check the frequency distribution of the encoded training pattern classification with encoded integers.

(unique, counts) = np.unique(y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[   0  913]
 [   1 2793]]


---
`Encode the textual features into series of vector of numbers`

In [14]:
# First get the word count vector of the pattern string to encode the pattern string.

tv = TfidfVectorizer()
x = tv.fit_transform(X)


In [15]:
# save the CountVectorizer to disk

joblib.dump(tv, 'presence_TfidfVectorizer.joblib')

['presence_TfidfVectorizer.joblib']

---
# Rough Idea about the effect of different classifiers
---

In [16]:
# Five models are tested:
# -- Logistic Regression
# -- Linear Support Vector Machine
# -- Random Forest
# -- Multinomial Naive Bayes
# -- Bernoulli Naive Bayes
# -- KNN

classifiers = [LogisticRegression(), LinearSVC(), RandomForestClassifier(), MultinomialNB(), BernoulliNB(), KNeighborsClassifier()]

In [17]:
# Calculate the accuracies of different classifiers using default settings.

acc = []
pre = []
cm = []

for clf in classifiers:
    y_pred = cross_val_predict(clf, x, y, cv=5, n_jobs = -1)
    acc.append(metrics.accuracy_score(y, y_pred))
    pre.append(metrics.precision_score(y,y_pred, pos_label=0))
    cm.append(metrics.confusion_matrix(y, y_pred))

In [18]:
# List the accuracies of different classifiers.

for i in range(len(classifiers)):
    print("{} accuracy: {:.3f}".format(classifiers[i],acc[i]))
    print("{} precision: {:.3f}".format(classifiers[i],pre[i]))
    print("Confusion Matrix: {}".format(cm[i]))

LogisticRegression() accuracy: 0.917
LogisticRegression() precision: 0.981
Confusion Matrix: [[ 619  294]
 [  12 2781]]
LinearSVC() accuracy: 0.956
LinearSVC() precision: 0.954
Confusion Matrix: [[ 787  126]
 [  38 2755]]
RandomForestClassifier() accuracy: 0.953
RandomForestClassifier() precision: 0.952
Confusion Matrix: [[ 777  136]
 [  39 2754]]
MultinomialNB() accuracy: 0.949
MultinomialNB() precision: 0.920
Confusion Matrix: [[ 792  121]
 [  69 2724]]
BernoulliNB() accuracy: 0.938
BernoulliNB() precision: 0.880
Confusion Matrix: [[ 790  123]
 [ 108 2685]]
KNeighborsClassifier() accuracy: 0.868
KNeighborsClassifier() precision: 0.926
Confusion Matrix: [[ 460  453]
 [  37 2756]]


---
# Bernoulli Naive Bayes Classifier


---
### `Use default setting of classifier hyperparameters`

In [19]:
clf_bnb = BernoulliNB()

In [20]:
y_pred = cross_val_predict(clf_bnb, x, y, cv=5, n_jobs = -1)

In [21]:
clf_bnb.get_params()

{'alpha': 1.0, 'binarize': 0.0, 'class_prior': None, 'fit_prior': True}

---
`use the default setting of hyperparameters of the Bernoulli Naive Bayes classifier`

In [22]:
print("Accuracy:", metrics.accuracy_score(y, y_pred))
print("Precision:", metrics.precision_score(y,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y, y_pred))

Accuracy: 0.9376686454398273
Precision: 0.8797327394209354
Confusion Matrix:
 [[ 790  123]
 [ 108 2685]]


In [23]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,  898],
       [   1, 2808]])

---
### `Parameter Tunning of BernoulliNB classifier`
`Define the combination of parameters to be considered`

In [24]:
param_grid = {'alpha':[0,1], 
              'fit_prior':[True, False]}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [25]:
gs = GridSearchCV(clf_bnb,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [26]:
best_bnb = gs.fit(x,y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.5s finished


In [27]:
scores_df = pd.DataFrame(best_bnb.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_alpha', 'param_fit_prior']]

,rank_test_score,mean_test_score,param_alpha,param_fit_prior
0,1,0.937666,1,True
1,2,0.930383,1,False
2,3,0.921750,0,True
3,4,0.880740,0,False


In [28]:
best_bnb.best_params_

{'alpha': 1, 'fit_prior': True}

---
`Save the best BernoulliNB model for future use`

In [29]:
# save the model to local disk

joblib.dump(best_bnb, 'bnb_presence_classifier.joblib')

['bnb_presence_classifier.joblib']

---
# Random Forest Classifier


---
### `Use default setting of classifier hyperparameters`

In [41]:
clf_rf = RandomForestClassifier()

In [42]:
y_pred = cross_val_predict(clf_rf, x, y, cv=5, n_jobs = -1)

In [43]:
clf_rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

---
`use the default setting of hyperparameters of the Random Forest classifier.`

In [44]:
print("Accuracy:", metrics.accuracy_score(y, y_pred))
print("Precision:", metrics.precision_score(y,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y, y_pred))

Accuracy: 0.9522396114409066
Precision: 0.9498777506112469
Confusion Matrix:
 [[ 777  136]
 [  41 2752]]


In [45]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,  818],
       [   1, 2888]])

---
### `Parameter Tunning of Random Forest classifier`
`Define the combination of parameters to be considered`

In [46]:
param_grid = {'bootstrap':[True,False], 
              'criterion':['gini','entropy'],
              'max_depth':[10,20,30,40,50,60,70,80,90,100, None],
              'min_samples_leaf':[1,2,4],
              'min_samples_split':[2,5,10],
              'n_estimators':[100,200,300,400,500,600]}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [47]:
gs = GridSearchCV(clf_rf,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [48]:
best_rf = gs.fit(x,y)

Fitting 5 folds for each of 2376 candidates, totalling 11880 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   35.0s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed: 21.3min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed: 30.0min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 35.8min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 46.4min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed: 59.9min
[Parallel(n_jobs=-1)]: Done 7176 tasks      | elapsed: 71.3min
[Parallel(n_jobs=-1)]: Done 8426 tasks      | elapsed: 97.0min
[Parallel(n_jobs=-1)]: Done 9776 tasks      | elapsed: 117.0min
[Parallel(n_jobs=-1)]: Done 11226 tasks     

In [49]:
scores_df = pd.DataFrame(best_rf.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_bootstrap', 'param_criterion','param_max_depth','param_min_samples_leaf','param_min_samples_split','param_n_estimators']]

,rank_test_score,mean_test_score,param_bootstrap,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators
0,1,0.959254,False,gini,None,2,5,100
1,2,0.959253,False,entropy,None,2,5,500
2,3,0.958982,False,entropy,80,1,5,100
3,4,0.958715,True,entropy,90,2,10,100
4,5,0.958444,False,gini,None,2,10,600
...,...,...,...,...,...,...,...,...
2371,2372,0.811651,True,entropy,10,1,5,100
2372,2373,0.811386,False,gini,10,2,10,200
2373,2374,0.810580,False,entropy,10,2,2,100
2374,2375,0.807878,True,gini,10,1,10,100


In [50]:
best_rf.best_params_

{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': None,
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 100}

---
`Save the best Random Forest model for future use`

In [51]:
# save the model to local disk

joblib.dump(best_rf, 'rf_presence_classifier.joblib')

['rf_presence_classifier.joblib']

---
# SVM Classifier


---
### `Use default setting of classifier hyperparameters`

In [30]:
clf_svm = LinearSVC()

In [31]:
y_pred = cross_val_predict(clf_svm, x, y, cv=5, n_jobs = -1)

In [32]:
clf_svm.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

---
`use the default setting of hyperparameters of the Random Forest classifier.`

In [33]:
print("Accuracy:", metrics.accuracy_score(y, y_pred))
print("Precision:", metrics.precision_score(y,y_pred, pos_label=0))
print("Confusion Matrix:\n", metrics.confusion_matrix(y, y_pred))

Accuracy: 0.9557474365893146
Precision: 0.953939393939394
Confusion Matrix:
 [[ 787  126]
 [  38 2755]]


In [34]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[   0,  825],
       [   1, 2881]])

---
### `Parameter Tunning of SVM classifier`
`Define the combination of parameters to be considered`

In [35]:
param_grid = {'C':[0.1,1,10,100],
              'penalty':['l1','l2']}

`Run the Grid Search`

Use cross validation on the training dataset to find optimal model.

In [36]:
gs = GridSearchCV(clf_svm,param_grid,cv=5, 
                      verbose = 1, n_jobs = -1)

In [37]:
best_svm = gs.fit(x,y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.1s finished


In [38]:
scores_df = pd.DataFrame(best_svm.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_penalty', 'param_C']]

,rank_test_score,mean_test_score,param_penalty,param_C
0,1,0.956830,l2,10
1,2,0.955750,l2,1
2,3,0.954402,l2,100
3,4,0.926879,l2,0.1
4,5,NaN,l1,0.1
5,6,NaN,l1,1
6,7,NaN,l1,10
7,8,NaN,l1,100


In [39]:
best_svm.best_params_

{'C': 10, 'penalty': 'l2'}

---
`Save the best SVM model for future use`

In [40]:
# save the model to local disk

joblib.dump(best_svm, 'svm_presence_classifier.joblib')

['svm_presence_classifier.joblib']